# Setup notebook

In [ ]:
import os
import sys
from datetime import datetime

import numpy as np
import pandas as pd

import keras

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import RandomTranslation

import matplotlib.pyplot as plt

%matplotlib inline


seed = 47
np.random.seed(seed)
tf.random.set_seed(seed)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tfk.__version__}")
print(f"GPU devices: {len(tf.config.list_physical_devices('GPU'))}")
BATCH_SIZE = 64

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
tf.get_logger().setLevel('ERROR')
sys.stderr = open(os.devnull, 'w')

TensorFlow version: 2.16.1
Keras version: 3.3.3
GPU devices: 1


I0000 00:00:1733312161.520030      88 service.cc:145] XLA service 0x7ca0a00035b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733312161.520079      88 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1733312188.508997      88 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


# Load and prepare data

In [ ]:
training_data = np.load("/kaggle/input/datasetlomi/training_set_no_outliers.npz")
test_data = np.load("/kaggle/input/datasetlomi/test_set.npz")
images = training_data["images"]/255
labels = training_data["labels"]
print(images.shape)

(2270, 64, 128)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2)

In [ ]:
def add_channel(image, label):
    image = tf.cast(image, tf.float32)
    label = tf.cast(label, tf.float32)
    image = tf.expand_dims(image, axis=-1)
    label = tf.expand_dims(label, axis=-1)
    return image, label

In [ ]:
translate_x_0_2_pos = keras.layers.RandomTranslation(height_factor=(0.0,0.0), width_factor=(0.3,0.3), fill_mode="constant", fill_value=0.0)
translate_x_0_2_neg = keras.layers.RandomTranslation(height_factor=(0.0,0.0), width_factor=(-0.3,-0.3), fill_mode="constant", fill_value=0.0)
translate_y_0_2_pos = keras.layers.RandomTranslation(height_factor=(0.3,0.3), width_factor=(0.0,0.0), fill_mode="constant", fill_value=0.0)
translate_y_0_2_neg = keras.layers.RandomTranslation(height_factor=(-0.3,-0.3), width_factor=(0.0,0.0), fill_mode="constant", fill_value=0.0)

@tf.function
def translate(image, label, seed):
    which_translation = tf.random.uniform([], seed=seed)
    return tf.case(
        [
            (which_translation < 0.25, lambda: (translate_x_0_2_pos(image), translate_x_0_2_pos(label))),
            (tf.logical_and(which_translation >= 0.25, which_translation < 0.5), lambda: (translate_x_0_2_neg(image), translate_x_0_2_neg(label))),
            (tf.logical_and(which_translation >= 0.5, which_translation < 0.75), lambda: (translate_y_0_2_pos(image), translate_y_0_2_pos(label))),
            (which_translation >= 0.75, lambda: (translate_y_0_2_neg(image), translate_y_0_2_neg(label))),
        ],
        default=lambda: (image,label),
        exclusive=True
    )

In [ ]:
@tf.function
def random_flip(image, label, seed=None):
    flip_prob = tf.random.uniform([], seed=seed)

    image = tf.cond(
        flip_prob > 1,
        lambda: tf.image.flip_left_right(image),
        lambda: image
    )
    label = tf.cond(
        flip_prob > 1,
        lambda: tf.image.flip_left_right(label),
        lambda: label
    )

    return image, label

@tf.function
def random_traslate(image, label, seed=None):
    flip_prob = tf.random.uniform([], seed=seed)

    image, label = tf.cond(
        flip_prob > 0,
        lambda: translate(image, label, seed),
        lambda: (image, label)
    )

    return image, label

In [ ]:
def to_datasett(X_train, y_train, augmentation = False, seed = seed, shuffle = False, batch_size = BATCH_SIZE):
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=batch_size * 2, seed=seed)

    dataset = dataset.map(
                    lambda x, y: add_channel(x, y),
                    num_parallel_calls=tf.data.AUTOTUNE
                   )

    if augmentation:
        dataset = dataset.map(
                        lambda x, y: random_traslate(x,y,seed),
                        num_parallel_calls=tf.data.AUTOTUNE
                    )


    """if augmentation:
        dataset = dataset.map(
                        lambda x, y: (tf.squeeze(x, axis=-1), tf.squeeze(y, axis=-1)),
                        num_parallel_calls=tf.data.AUTOTUNE
                    )"""
    # Batch the data
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [ ]:
train_dataset = to_datasett(X_train, y_train, augmentation = True)
#train_dataset_no_aug = to_datasett(X_train, y_train, augmentation = False)

#train_dataset = train_dataset_aug.concatenate(train_dataset_no_aug)

val_dataset = to_datasett(X_val, y_val, augmentation=False)

# Model definition

In [ ]:
num_classes = 5
epochs = 1000
patience = 20
complexLoss = 4
batch_size = 64
seed = 45
learning_rate = 0.001
patience = 20

In [ ]:
def unetpp_block(inputs, filters, name):
    """
    Crea un nodo della U-Net++ che combina input provenienti da diversi livelli.
    """
    if len(inputs) > 1:
        x = tfkl.Concatenate(name=f"{name}_concat")(inputs)
    else:
        x = inputs[0]

    # Primo blocco convoluzionale con LeakyReLU
    x = tfkl.Conv2D(filters, kernel_size=3, padding="same", name=f"{name}_conv1")(x)
    x = tfkl.BatchNormalization(name=f"{name}_bn1")(x)
    x = tfkl.LeakyReLU(negative_slope=0.1, name=f"{name}_leakyrelu1")(x)  # Sostituito alpha con negative_slope

    # Secondo blocco convoluzionale con LeakyReLU
    x = tfkl.Conv2D(filters, kernel_size=3, padding="same", name=f"{name}_conv2")(x)
    x = tfkl.BatchNormalization(name=f"{name}_bn2")(x)
    x = tfkl.LeakyReLU(negative_slope=0.1, name=f"{name}_leakyrelu2")(x)  # Sostituito alpha con negative_slope

    return x


In [ ]:
def get_unetpp(input_shape=(64, 128,1), num_classes=5, filters=[32, 64, 128, 256, 512], seed=seed):
    tf.random.set_seed(seed)
    input_layer = tfkl.Input(shape=input_shape, name="input_layer")

    # Downsampling path
    x00 = unetpp_block([input_layer], filters[0], name="x00")
    d0 = tfkl.MaxPooling2D(pool_size=(2, 2))(x00)

    x10 = unetpp_block([d0], filters[1], name="x10")
    d1 = tfkl.MaxPooling2D(pool_size=(2, 2))(x10)

    x20 = unetpp_block([d1], filters[2], name="x20")
    d2 = tfkl.MaxPooling2D(pool_size=(2, 2))(x20)

    x30 = unetpp_block([d2], filters[3], name="x30")
    d3 = tfkl.MaxPooling2D(pool_size=(2, 2))(x30)

    x40 = unetpp_block([d3], filters[4], name="x40")

    # Upsampling path with dense connections
    x01 = unetpp_block([x00, tfkl.UpSampling2D(size=(2, 2))(x10)], filters[0], name="x01")
    x11 = unetpp_block([x10, tfkl.UpSampling2D(size=(2, 2))(x20)], filters[1], name="x11")
    x02 = unetpp_block([x01, tfkl.UpSampling2D(size=(2, 2))(x11)], filters[0], name="x02")

    x21 = unetpp_block([x20, tfkl.UpSampling2D(size=(2, 2))(x30)], filters[2], name="x21")
    x12 = unetpp_block([x11, tfkl.UpSampling2D(size=(2, 2))(x21)], filters[1], name="x12")
    x03 = unetpp_block([x02, tfkl.UpSampling2D(size=(2, 2))(x12)], filters[0], name="x03")

    x31 = unetpp_block([x30, tfkl.UpSampling2D(size=(2, 2))(x40)], filters[3], name="x31")
    x22 = unetpp_block([x21, tfkl.UpSampling2D(size=(2, 2))(x31)], filters[2], name="x22")
    x13 = unetpp_block([x12, tfkl.UpSampling2D(size=(2, 2))(x22)], filters[1], name="x13")
    x04 = unetpp_block([x03, tfkl.UpSampling2D(size=(2, 2))(x13)], filters[0], name="x04")

    # Output Layer
    output_layer = tfkl.Conv2D(num_classes, kernel_size=1, activation="softmax", name="output_layer")(x04)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name="UNetPlusPlus")
    return model

In [ ]:
#model = get_unetpp()

# Print a detailed summary of the model with expanded nested layers and trainable parameters.
#model.summary(expand_nested=True, show_trainable=True)

# Generate and display a graphical representation of the model architecture.
#tf.keras.utils.plot_model(model, show_trainable=True, expand_nested=True, dpi=70)

# **MeanIntersectionOverUnion**

In [ ]:
class MeanIntersectionOverUnion(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes, labels_to_exclude=[0], name="mean_iou", dtype=None, **kwargs):
        """
        Aggiunto **kwargs per gestire parametri inattesi come `ignore_class`.
        """
        super(MeanIntersectionOverUnion, self).__init__(num_classes=num_classes, name=name, dtype=dtype)
        if labels_to_exclude is None:
            labels_to_exclude = [0]
        self.labels_to_exclude = labels_to_exclude

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.math.argmax(y_pred, axis=-1)
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.reshape(y_pred, [-1])

        for label in self.labels_to_exclude:
            mask = tf.not_equal(y_true, label)
            y_true = tf.boolean_mask(y_true, mask)
            y_pred = tf.boolean_mask(y_pred, mask)

        return super().update_state(y_true, y_pred, sample_weight)

# Registra la classe personalizzata
tf.keras.utils.get_custom_objects()["MeanIntersectionOverUnion"] = MeanIntersectionOverUnion

# **weighted_loss**

In [ ]:
labels_flat = y_train.reshape(-1)  # Appiattisci l'array per considerare tutti i pixel

# Conta la frequenza di ogni classe
num_classes = 5  # Modifica in base al numero di classi nel tuo dataset
labels_flat = labels_flat.astype(int)  # Converte i dati in interi

# Conta la frequenza di ogni classe
class_frequencies = np.bincount(labels_flat, minlength=num_classes)

# Stampa le frequenze
print(f"Frequenze delle classi: {class_frequencies}")

total_samples = np.sum(class_frequencies)
class_weights = total_samples / (len(class_frequencies) * class_frequencies)
print(f"Pesi di classe: {class_weights}")

# Conversione in dizionario per TensorFlow
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Funzione di perdita con pesi di classe
loss = tf.keras.losses.SparseCategoricalCrossentropy()


def weighted_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.int32)  # Assicurati che i valori siano interi
    weights = tf.gather(class_weights, y_true)  # Recupera i pesi in base alle etichette
    weights = tf.cast(weights, tf.float32)  # Converte i pesi in float32
    unweighted_loss = loss(y_true, y_pred)
    weighted_loss = unweighted_loss * weights
    return tf.reduce_mean(weighted_loss)

Frequenze delle classi: [3919495 4121335 3893379 2921642   20821]
Pesi di classe: [  0.75911167   0.72193462   0.76420364   1.01837747 142.90064838]


# **Dice score**

In [ ]:
class DiceScore(tf.keras.metrics.Metric):
    def __init__(self, name="DiceScore", smooth=1e-6, **kwargs):
        super(DiceScore, self).__init__(name=name, **kwargs)
        self.smooth = smooth
        self.true_positive = self.add_weight(name="true_positive", initializer="zeros")
        self.pred_positive = self.add_weight(name="pred_positive", initializer="zeros")
        self.actual_positive = self.add_weight(name="actual_positive", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Rimuovi il canale dal tensor y_true (se presente)
        y_true = tf.squeeze(y_true, axis=-1)

        # Calcola le etichette predette come argmax
        y_pred = tf.argmax(y_pred, axis=-1)
        y_pred = tf.cast(y_pred, tf.float32)  # Assicurati che sia float32

        # Calcola l'intersezione e le somme
        intersection = tf.reduce_sum(y_true * y_pred)
        pred_sum = tf.reduce_sum(y_pred)
        true_sum = tf.reduce_sum(y_true)

        # Aggiorna gli stati
        self.true_positive.assign_add(intersection)
        self.pred_positive.assign_add(pred_sum)
        self.actual_positive.assign_add(true_sum)

    def result(self):
        numerator = 2 * self.true_positive + self.smooth
        denominator = self.pred_positive + self.actual_positive + self.smooth
        return numerator / denominator

    def reset_states(self):
        self.true_positive.assign(0)
        self.pred_positive.assign(0)
        self.actual_positive.assign(0)

tf.keras.utils.get_custom_objects()["DiceScore"] = DiceScore

In [ ]:
class DiceLoss(tf.keras.losses.Loss):
    def __init__(self, smooth=1e-6, name="DiceLoss"):
        super(DiceLoss, self).__init__(name=name)
        self.smooth = smooth

    def call(self, y_true, y_pred):
        # Rimuovi la dimensione del canale da y_true (se necessario)
        y_true = tf.squeeze(y_true, axis=-1)

        # Converti y_pred in probabilità (softmax già applicato nei modelli multi-classe)
        y_pred = tf.argmax(y_pred, axis=-1)
        y_pred = tf.cast(y_pred, tf.float32)  # Assicurati che sia float32

        # Calcolo dell'intersezione e della somma
        intersection = tf.reduce_sum(y_true * y_pred)
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)

        # Calcolo del Dice Score
        dice_score = (2. * intersection + self.smooth) / (union + self.smooth)

        # La perdita è 1 - Dice Score
        dice_loss = 1 - dice_score
        return dice_loss

    def get_config(self):
        return {"smooth": self.smooth, "name": self.name}

# **FocalLoss**

In [ ]:
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=0.25, name="FocalLoss"):
        super(FocalLoss, self).__init__(name=name)
        self.gamma = gamma
        self.alpha = alpha

    def call(self, y_true, y_pred):
        # Clip y_pred per evitare log(0)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0)

        # Calcolo della focal loss
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = self.alpha * tf.math.pow(1 - y_pred, self.gamma)
        focal_loss = tf.reduce_sum(weight * cross_entropy, axis=-1)
        return tf.reduce_mean(focal_loss)

    def get_config(self):
        return {"gamma": self.gamma, "alpha": self.alpha, "name": self.name}

# **BoundaryLoss**

In [ ]:
from tensorflow.python.keras import backend as K

def compute_signed_distance_map(mask):
    # Creiamo un kernel che emula l'effetto di una convoluzione che calcola la distanza ai bordi
    # Questo è un esempio di kernel semplice per la distanza
    # Aggiungi un canale per la convoluzione, se necessario
    mask_expanded = tf.expand_dims(mask, axis=-1)  # Forma: [batch_size, height, width, 1]

    # Definiamo il kernel per la mappa di distanza (3x3)
    kernel = tf.convert_to_tensor([[ [1], [1], [1]],
                                    [ [1], [-7], [1]],
                                    [ [1], [1], [1]]], dtype=tf.float32)

    kernel = tf.reshape(kernel, (3, 3, 1, 1))  # Forma del kernel: [3, 3, 1, 1]

    # Applichiamo la convoluzione 2D alla maschera
    distance_map = tf.nn.conv2d(mask_expanded, kernel, strides=[1, 1, 1, 1], padding='SAME')

    # Normalizziamo la distanza (valore assoluto)
    distance_map = tf.abs(distance_map)

    return distance_map

# Definizione della Boundary Loss
class BoundaryLoss(tf.keras.losses.Loss):
    def __init__(self, name="BoundaryLoss"):
        super(BoundaryLoss, self).__init__(name=name)

    def call(self, y_true, y_pred):
        OUTPUT_SHAPE = (64, 128, 1)
        y_pred_bd = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_pred)
        y_true_bd = tfkl.MaxPooling2D((3, 3), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_true)
        y_pred_bd = y_pred_bd - (1 - y_pred)
        y_true_bd = y_true_bd - (1 - y_true)

        y_pred_bd_ext = tfkl.MaxPooling2D((5, 5), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_pred)
        y_true_bd_ext = tfkl.MaxPooling2D((5, 5), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_true)
        y_pred_bd_ext = y_pred_bd_ext - (1 - y_pred)
        y_true_bd_ext = y_true_bd_ext - (1 - y_true)

        P = K.sum(y_pred_bd * y_true_bd_ext) / K.sum(y_pred_bd) + 1e-7
        R = K.sum(y_true_bd * y_pred_bd_ext) / K.sum(y_true_bd) + 1e-7
        F1_Score = 2 * P * R / (P + R + 1e-7)
        # print(f'Precission: {P.eval()}, Recall: {R.eval()}, F1: {F1_Score.eval()}')
        loss = K.mean(1 - F1_Score)
        # print(f"Loss:{loss.eval()}")
        return loss

    def get_config(self):
        return {"name": self.name}

# **Loss combination**

In [ ]:
class UnifiedLoss(tf.keras.losses.Loss):
    def __init__(self, dice_weight=0.5, focal_weight=0.3, boundary_weight=0.2,
                 gamma=2.0, alpha=0.25, smooth=1e-6, name="UnifiedLoss"):
        super(UnifiedLoss, self).__init__(name=name)
        self.dice_loss = DiceLoss(smooth=smooth)
        self.focal_loss = FocalLoss(gamma=gamma, alpha=alpha)
        self.boundary_loss = BoundaryLoss()
        self.dice_weight = dice_weight
        self.focal_weight = focal_weight
        self.boundary_weight = boundary_weight

    def call(self, y_true, y_pred):
        dice = self.dice_loss(y_true, y_pred)
        focal = self.focal_loss(y_true, y_pred)
        boundary = self.boundary_loss(y_true, y_pred)
        return (self.dice_weight * dice +
                self.focal_weight * focal +
                self.boundary_weight * boundary)

    def get_config(self):
        return {
            "dice_weight": self.dice_weight,
            "focal_weight": self.focal_weight,
            "boundary_weight": self.boundary_weight,
            "gamma": self.gamma,
            "alpha": self.alpha,
            "smooth": self.smooth,
            "name": self.name
        }

tf.keras.utils.get_custom_objects()["UnifiedLoss"] = UnifiedLoss

# **Per la visualizzazione durante il trainig**

In [ ]:
def create_segmentation_colormap(num_classes):
    """
    Create a linear colormap using a predefined palette.
    Uses 'viridis' as default because it is perceptually uniform
    and works well for colorblindness.
    """
    return plt.cm.viridis(np.linspace(0, 1, num_classes))

def apply_colormap(label, colormap=None):
    """
    Apply the colormap to a label.
    """
    # Ensure label is 2D
    label = np.squeeze(label)

    if colormap is None:
        num_classes = len(np.unique(label))
        colormap = create_segmentation_colormap(num_classes)

    # Apply the colormap
    colored = colormap[label.astype(int)]

    return colored

class VizCallback(tf.keras.callbacks.Callback):
    def __init__(self, dataset, frequency=5, num_classes=2):
        super().__init__()
        self.dataset = dataset
        self.frequency = frequency
        self.num_classes = num_classes
        self.dataset_iter = iter(dataset)  # Crea un iteratore per accedere ai dati

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.frequency == 0:  # Visualizza solo ogni "frequency" epochs
            try:
                # Estrai un batch di dati
                image, label = next(self.dataset_iter)
            except StopIteration:
                # Ricrea l'iteratore se i dati sono terminati
                self.dataset_iter = iter(self.dataset)
                image, label = next(self.dataset_iter)

            # Prepara i dati per la predizione
            image = tf.expand_dims(image[0], 0)  # Estrai una sola immagine dal batch
            label = label[0]  # Etichetta corrispondente
            pred = self.model.predict(image, verbose=0)
            y_pred = tf.math.argmax(pred, axis=-1)
            y_pred = y_pred.numpy()

            # Creazione della mappa colori
            colormap = create_segmentation_colormap(self.num_classes)

            plt.figure(figsize=(16, 4))

            # Immagine di input
            plt.subplot(1, 3, 1)
            plt.imshow(image[0])
            plt.title("Input Image")
            plt.axis('off')

            # Ground truth
            plt.subplot(1, 3, 2)
            colored_label = apply_colormap(label.numpy(), colormap)
            plt.imshow(colored_label)
            plt.title("Ground Truth Mask")
            plt.axis('off')

            # Predizione
            plt.subplot(1, 3, 3)
            colored_pred = apply_colormap(y_pred[0], colormap)
            plt.imshow(colored_pred)
            plt.title("Predicted Mask")
            plt.axis('off')

            plt.tight_layout()
            plt.show()
            plt.close()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=patience,
    restore_best_weights=True
)

#  **Cross validation**

In [ ]:
from sklearn.model_selection import KFold

# Define the function for cross-validation using KFold
def cross_validate(n_splits=5, batch_size=batch_size, dice_weight=0.5, focal_weight=0.3, boundary_weight=0.2):
    # Initialize KFold with the specified number of splits and shuffling
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    fold_no = 1  # Track the fold number
    val_losses = []  # Store validation losses for each fold
    val_accuracies = []  # Store validation accuracies for each fold

    # Extract all images and labels from the training dataset
    # Assumes `train_dataset` contains image and label pairs
    images = []
    labels = []
    for image, label in train_dataset:
        images.append(image.numpy())
        labels.append(label.numpy())

    # Convert lists of images and labels into numpy arrays
    images = np.concatenate(images, axis=0)
    labels = np.concatenate(labels, axis=0)

    # Perform KFold cross-validation
    for train_idx, val_idx in kfold.split(images, labels):
        # Split the data into training and validation sets
        X_train, X_val = images[train_idx], images[val_idx]
        y_train, y_val = labels[train_idx], labels[val_idx]

        # Create TensorFlow datasets for training and validation
        train_dataset_fold = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
        val_dataset_fold = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

        # Initialize a new model for this fold
        model = get_unetpp()

        # Compile the model with the specified loss and metrics
        model.compile(
            loss=UnifiedLoss(dice_weight=dice_weight, focal_weight=focal_weight, boundary_weight=boundary_weight),
            optimizer=tf.keras.optimizers.AdamW(learning_rate=learning_rate),
            metrics=[
                "accuracy",  # Standard accuracy metric
                DiceScore(),  # Custom Dice Score metric
                MeanIntersectionOverUnion(num_classes=num_classes, labels_to_exclude=[0])  # Mean IoU ignoring label 0
            ]
        )

        # Train the model on the current fold
        history = model.fit(
            train_dataset_fold,
            epochs=epochs,
            validation_data=val_dataset_fold,
            callbacks=[early_stopping],
            verbose=0  # Suppress verbose output
        )

        # Extract the final validation loss and accuracy
        val_loss = history.history['val_loss'][-1]
        val_accuracy = history.history.get('val_accuracy', [None])[-1]

        # Append the results to their respective lists
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        # Delete the model to free memory
        del model

        # Early exit if accuracy is too low
        if val_accuracy < 0.5:
            print("Drop the training, too slow accuracy. Continue with next set of values.")
            break

        print(f"Fold {fold_no} - Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        fold_no += 1

    # Calculate the average validation loss and accuracy across all folds
    avg_val_loss = np.mean(val_losses)
    avg_val_accuracy = np.mean(val_accuracies)

    print(f"\nAverage Validation Loss: {avg_val_loss:.4f}")
    print(f"Average Validation Accuracy: {avg_val_accuracy:.4f}")

    return avg_val_loss, avg_val_accuracy

# Define parameter values for Grid Search
dice_weight_vals = [0.2, 0.3, 0.4, 0.5]
focal_weight_vals = [0.1, 0.2, 0.3]
boundary_weight_vals = [0.1, 0.2, 0.3]

best_loss = float('inf')  # Initialize the best loss
best_params = {}  # Store the best parameters

# Iterate through all combinations of parameters for Grid Search
for dice_weight in dice_weight_vals:
    for focal_weight in focal_weight_vals:
        for boundary_weight in boundary_weight_vals:
            print(f"\nTesting dice_weight={dice_weight}, focal_weight={focal_weight}, boundary_weight={boundary_weight}")

            # Perform cross-validation with the current parameter combination
            avg_loss, avg_accuracy = cross_validate(dice_weight=dice_weight, focal_weight=focal_weight, boundary_weight=boundary_weight)

            # Update the best parameters if a better loss is found
            if avg_loss < best_loss:
                best_loss = avg_loss
                best_params = {
                    'dice_weight': dice_weight,
                    'focal_weight': focal_weight,
                    'boundary_weight': boundary_weight
                }

print(f"\nBest parameters: {best_params}")



Testing dice_weight=0.2, focal_weight=0.1, boundary_weight=0.1
Drop the training, to slow accuracy. Continue with next set of values

Average Validation Loss: 0.0846
Average Validation Accuracy: 0.1960

Testing dice_weight=0.2, focal_weight=0.1, boundary_weight=0.2
Drop the training, to slow accuracy. Continue with next set of values

Average Validation Loss: 0.2043
Average Validation Accuracy: 0.0022

Testing dice_weight=0.2, focal_weight=0.1, boundary_weight=0.3
Drop the training, to slow accuracy. Continue with next set of values

Average Validation Loss: 0.3184
Average Validation Accuracy: 0.1974

Testing dice_weight=0.2, focal_weight=0.2, boundary_weight=0.1
Drop the training, to slow accuracy. Continue with next set of values

Average Validation Loss: 0.2657
Average Validation Accuracy: 0.2158

Testing dice_weight=0.2, focal_weight=0.2, boundary_weight=0.2
Drop the training, to slow accuracy. Continue with next set of values

Average Validation Loss: 0.3136
Average Validation Ac